In [32]:
import lzma
import xml.etree.ElementTree as ET

file_path = r"../sample_problems/sample_problem_test/AircraftLanding-table-airland01_mc22.xml"
# with lzma.open(file_path, "rb") as compressed_file:
#     xml_bytes = compressed_file.read()
with open(file_path, "rb") as bytes:
    root = ET.fromstring(xml_bytes)

In [33]:
variables = root.findall("variables")
constraints = root.findall("constraints")
objectives = root.findall("objectives")

array_vars = variables[0].findall("array")
integer_vars = variables[0].findall("var")

In [28]:
import ast
import numpy as np
from typing import List


class Variable:
    """
    Attributes:
    - name (str): variable's name
    - domain (List[List[int]]): all intervals comprising the variable's domain; ex: [[1,5], [8], [11, 16]] means that the variable's domain is {1, 2, 3, 4, 5, 8, 11, 12, 13, 14, 15, 16}
    """
    def __init__(self, name:str, domain:List[List[int]]):
        self.name = name
        self.domain = domain
    def __repr__(self):
        return repr(f"""Name: {self.name} Domain: {self.domain}""")


def get_array_dimensions(size):
    dimension_sizes = ast.literal_eval("[" + size.replace("]", ",").replace("[", "") + "]")
    return dimension_sizes

def parse_variable_domain(raw_domain):
    domain = []
    sub_domains = [dom_element for dom_element in raw_domain.replace("..", ",").split(" ")]
    for sub_domain in sub_domains:
        if not sub_domain:
            continue
        if "," in sub_domain:
            sub_domain = sub_domain.split(",")
            domain.append([int(sub_domain[0]), int(sub_domain[1])])
        else:
            domain.append([int(sub_domain)])
    return domain

def build_variable(variable_name, current_position, domain):
    full_variable_name = variable_name[0] + str(current_position)
    new_var = Variable(full_variable_name, domain)
    
    return new_var

def parse_variable(var_name, dim_index, dimensions, current_position, domain, variables, sizes, allocated_variables):
    if dim_index == len(dimensions):
        new_variable = build_variable(var_name, current_position, domain)
        variables.append(new_variable)
        allocated_variables[tuple(current_position)] = 1
        return
    current_dim = dimensions[dim_index]
    if current_dim:
        if ".." in current_dim:
            dims_range = [int(dim) for dim in current_dim.split("..")]
            lower_bound = dims_range[0]
            upper_bound = dims_range[1] + 1
        else:
            lower_bound = int(current_dim)
            upper_bound = int(current_dim) + 1
        for dim in range(lower_bound, upper_bound):
            current_position.append(dim)
            parse_variable(var_name, dim_index+1, dimensions, current_position, domain, variables, sizes, allocated_variables)
            current_position.pop()
    else:
        for dim in range(sizes[dim_index]):
            current_position.append(dim)
            parse_variable(var_name, dim_index+1, dimensions, current_position, domain, variables, sizes, allocated_variables)
            current_position.pop()

instance_variables = []
for array in array_vars:
    array_dimensions = get_array_dimensions(array.attrib["size"])
    array_domains_parsed = np.zeros(array_dimensions)
    for variable in array:
        if variable.tag == "domain":
            domain = parse_variable_domain(variable.text)
            var_names = variable.attrib["for"].split()
            for new_var_name in var_names:
                current_array_dims = new_var_name.replace("]", "").split('[')[1:]
                parse_variable(new_var_name, 0, current_array_dims, [], domain, instance_variables, array_dimensions, array_domains_parsed)


In [29]:
instance_variables[0]

'Name: x[0] Domain: [[129, 559]]'

In [23]:
for var in instance_variables:
    print(var)

'Name: x[0] \n Domain: [[129, 559]]'
'Name: x[1] \n Domain: [[195, 744]]'
'Name: x[2] \n Domain: [[89, 510]]'
'Name: x[3] \n Domain: [[96, 521]]'
'Name: x[4] \n Domain: [[110, 555]]'
'Name: x[5] \n Domain: [[120, 576]]'
'Name: x[6] \n Domain: [[124, 577]]'
'Name: x[7] \n Domain: [[126, 573]]'
'Name: x[8] \n Domain: [[135, 591]]'
'Name: x[9] \n Domain: [[160, 657]]'
'Name: e[0] \n Domain: [[0, 26]]'
'Name: e[1] \n Domain: [[0, 63]]'
'Name: e[2] \n Domain: [[0, 9]]'
'Name: e[3] \n Domain: [[0, 10]]'
'Name: e[4] \n Domain: [[0, 13]]'
'Name: e[5] \n Domain: [[0, 15]]'
'Name: e[8] \n Domain: [[0, 15]]'
'Name: e[6] \n Domain: [[0, 14]]'
'Name: e[7] \n Domain: [[0, 14]]'
'Name: e[9] \n Domain: [[0, 20]]'
'Name: t[0] \n Domain: [[0, 404]]'
'Name: t[1] \n Domain: [[0, 486]]'
'Name: t[2] \n Domain: [[0, 412]]'
'Name: t[3] \n Domain: [[0, 415]]'
'Name: t[4] \n Domain: [[0, 432]]'
'Name: t[5] \n Domain: [[0, 441]]'
'Name: t[8] \n Domain: [[0, 441]]'
'Name: t[6] \n Domain: [[0, 439]]'
'Name: t[7] \